# Prompt Engineering 101 - Part IV.
## Advanced Reasoning Architectures

----

In [ ]:
# @title 🛠️ Step 1: Laboratory Setup (Gemini API with Recursive Wrapper)
# We are connecting to Google's "Gemini 2.5 Flash" model.

# 1. Install the Google AI SDK
!pip install -q -U google-genai

from google.colab import userdata
from google import genai
from google.genai import types
from IPython.display import display, Markdown

# 2. Configure the API Key
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# 3. Create a Robust Wrapper Class
class GeminiModel:
    def __init__(self, api_key, preferred_model='gemini-2.5-flash'):
        self.client = genai.Client(api_key=api_key)
        
        # Priority list of models and their availability status
        # True = Available, False = Exhausted (429)
        self.models = {
            'gemini-2.5-flash': True,
            'gemini-2.5-flash-lite': True,
            'gemini-3-flash-preview': True,
        }
        
        # Validation: Ensure the user's choice is valid
        if preferred_model not in self.models:
            raise ValueError(f"Invalid model '{preferred_model}'. "
                             f"Valid options: {list(self.models.keys())}")
            
        self.current_model = preferred_model

    def _get_next_available_model(self):
        """Sets the first model from the non-exhausted models as the currently selected model. 
        Raises error if no available model left."""
        for model_name, is_available in self.models.items():
            if is_available:
                print(f"🔄 Switching to fallback: {model_name}")
                self.current_model = model_name
                return 
    
        raise RuntimeError("❌ All available models are currently exhausted. "
                           "Please wait for quotas to reset.")

    def generate_content(self, contents, config=None):
        """
        Recursively attempts to generate content.
        If a model fails with a quota error, it marks it as unavailable 
        and retries with the next available model.
        """
        try:
            # Attempt generation
            response = self.client.models.generate_content(
                model=self.current_model,
                contents=contents,
                config=config
            )
            return response

        except Exception as e:
            # Check for Rate Limit / Quota errors
            if "429" in str(e) or "ResourceExhausted" in str(e):
                print(f"⚠️ Quota exceeded for {self.current_model}. Marking as unavailable...")
                
                # Update State: Mark current as failed
                self.models[self.current_model] = False
                
                # Switch to next available
                self._get_next_available_model()
            
                # Recursive Step: Try again with the new model
                return self.generate_content(contents, config)
            
            # If it's a logic error (e.g. invalid prompt), raise immediately
            raise e

# 4. Initialize
try:
    model = GeminiModel(GEMINI_API_KEY, preferred_model='gemini-2.5-flash')
    print(f"✅ Connection Established. Primary: {model.current_model}")
except Exception as e:
    print(f"❌ Error: {e}")

----

# 🏠 Homework: The "CEO's Brain"

### The Scenario
You are advising the CEO of a failing bookstore chain.
They have 3 options:
1.  Close all stores and go Online Only.
2.  Turn stores into "Coffee & Community" hubs.
3.  Sell the company to a larger retailer.

### The Task
Write a **Single Python Prompt** that uses a "System of Thought" to solve this.
Your prompt must include:
1.  **Persona:** Strategic Consultant.
2.  **Tree of Thoughts:** Analyze all 3 options (Pros/Cons/Risks).
3.  **Recursive Criticism:** Critique the initial analysis for bias.
4.  **Final Recommendation:** A synthesized conclusion.

### Submission
Submit the Python code and the final output.

In [ ]:
# @title (Hidden) Answer Key
advisor_prompt = """
ACT AS: Senior Strategy Consultant.
SCENARIO: Failing Bookstore Chain. Options: (1) Online Only, (2) Coffee Hubs, (3) Sell.

PROCESS:
1. TREE OF THOUGHTS: Analyze each of the 3 options. List the Financial Upside, Brand Risk, and Execution Difficulty for each.
2. CRITIQUE: Look at the analysis in Step 1. Are we being too optimistic about "Coffee Hubs"? Are we underestimating the cost of "Online Only"?
3. SYNTHESIS: Based on the critique, select the single best path forward and explain why.
"""